In [5]:
import numpy
import lsst.afw.table
import lsst.afw.image
import lsst.afw.detection
import lsst.meas.algorithms
import lsst.meas.base
import lsst.meas.deblender

In [6]:
import lsst.afw.display

In [7]:
schema = lsst.afw.table.SourceTable.makeMinimalSchema()
detectConfig = lsst.meas.algorithms.SourceDetectionConfig()
detectConfig.returnOriginalFootprints = False
detectConfig.thresholdValue = 10
detectTask = lsst.meas.algorithms.SourceDetectionTask(config=detectConfig, schema=schema)
deblendTask = lsst.meas.deblender.SourceDeblendTask(schema=schema)

In [8]:
measureConfig = lsst.meas.base.SingleFrameMeasurementConfig()
measureConfig.plugins.names.remove("base_GaussianCentroid")
import lsst.meas.modelfit
measureConfig.plugins.names |= ["modelfit_ShapeletPsfApprox", "modelfit_CModel"]
measureConfig.slots.modelFlux = "modelfit_CModel"
measureConfig.plugins["base_ClassificationExtendedness"].fluxRatio = 0.985
measureTask = lsst.meas.base.SingleFrameMeasurementTask(schema=schema)

In [9]:
exposure = lsst.afw.image.ExposureF("coadd.fits")

In [10]:
# Run detetion
table = lsst.afw.table.SourceTable.make(schema)
detectResult = detectTask.run(table, exposure)
catalog = detectResult.sources

In [11]:
# Remove the edge mask, since it just clutters up the figures.
mask = exposure.getMaskedImage().getMask()
mask.clearMaskPlane(mask.getMaskPlane("EDGE"))

In [14]:
# Display the post-detection image, and show all Peaks (Footprints automatically indicated by mask plane)
display = lsst.afw.display.getDisplay(frame=0)
display.setMaskTransparency(80)
display.mtv(exposure)
with display.Buffering():
    for record in catalog:
        for peak in record.getFootprint().getPeaks():
            display.dot("+", peak.getIx(), peak.getIy(), origin=lsst.afw.image.PARENT)
        print record.get("id"), record.get("parent"), record.get("deblend_nChild")

1 0 0
2 0 0
3 0 2
4 3 0
5 3 0


In [15]:
# Run deblender
deblendTask.run(exposure, catalog, psf=exposure.getPsf())

In [16]:
# Display the post-deblend HeavyFootprints
n = 1
for record in catalog:
    display = lsst.afw.display.getDisplay(frame=n)
    image = lsst.afw.image.ImageF(exposure.getBBox())
    image.getArray()[:,:] = numpy.nan
    footprint = record.getFootprint()
    if record.get("parent") == 0:
        array = numpy.zeros(footprint.getArea(), dtype=numpy.float32)
        lsst.afw.detection.flattenArray(footprint, exposure.getMaskedImage().getImage().getArray(),
                                        array, exposure.getXY0())
        lsst.afw.detection.expandArray(footprint, array, image.getArray(), image.getXY0())
    else:
        heavy = lsst.afw.detection.HeavyFootprintF.cast(footprint)
        heavy.insert(image)
    display.mtv(image)
    print record.get("id"), record.get("parent"), record.get("deblend_nChild")
    n += 1

1 0 0
2 0 0
3 0 2
4 3 0
5 3 0
6 3 0
7 3 0


In [17]:
# Run measurement
measureTask.run(catalog, exposure)

In [18]:
# Display the image with measured 2nd-moment ellipses
n = len(catalog) + 1
display = lsst.afw.display.getDisplay(frame=n)
display.setMaskTransparency(100)
display.mtv(exposure)
with display.Buffering():
    for record in catalog:
        display.dot(record.getShape(), record.getX(), record.getY(), origin=lsst.afw.image.PARENT)
n += 1

In [19]:
# Run NoiseReplacer and display the intermediate images
n = len(catalog) + 2
exposureCopy = exposure.clone()
fpDict = {record.getId(): (record.getParent(), record.getFootprint()) for record in catalog}
replacer = lsst.meas.base.NoiseReplacer(lsst.meas.base.NoiseReplacerConfig(), exposureCopy, fpDict)
display = lsst.afw.display.getDisplay(frame=n)
display.setMaskTransparency(100)
display.mtv(exposureCopy)
n += 1
for record in catalog:
    replacer.insertSource(record.getId())
    display = lsst.afw.display.getDisplay(frame=n)
    display.setMaskTransparency(100)
    display.mtv(exposureCopy)
    replacer.removeSource(record.getId())
    n += 1